<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/topic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic modelling for social network (VKontakte) posts

Resources:
*   [topic_model.ipynb](https://github.com/componavt/sns4human/blob/main/src/vk/nlp/topic_model.ipynb) (this file at GitHub sns4human)
*   [News topic modeling.ipynb](https://colab.research.google.com/drive/1p7mGJ5UNV5fViHP6HGQ6kQsjDe2-d5fN?usp=sharing#scrollTo=Ol7_oSFgzrQq)
*   [Пишем чат-бот на Python + PostgreSQL и Telegram](https://habr.com/ru/articles/593065/)
*   Бартеньев О. В. [Программирование моделей текста на Python](http://www.100byte.ru/python/text_models/text_models.html)
*   [Introduction to topic modeling.ipynb](https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/text-analysis/notebooks/Introduction%20to%20topic%20modeling.ipynb)
*   [Topic Modeling on Research Papers](https://colab.research.google.com/github/dipanjanS/nlp_workshop_odsc19/blob/master/Module05%20-%20NLP%20Applications/Project04%20-%20Topic%20Modeling.ipynb)
*   [pymorphy2 documentation](https://pymorphy2.readthedocs.io/en/0.2/user/index.html)
*   Шаг. 1. 4. [Предварительный анализ коллекции](https://colab.research.google.com/github/king-menin/mipt-nlp2022/blob/master/seminars/sem1/sem1_basic_text_processing.ipynb#scrollTo=-_3tbIdivvY8) (и гистограмма). Это фрагмент курса [mipt-nlp2022](https://github.com/king-menin/mipt-nlp2022/tree/master).

Our files (sns4human project):
*   [lemma_tag_cloud.ipynb](https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/lemma_tag_cloud.ipynb)
*   [vk_subscribers_research.ipynb](https://github.com/componavt/sns4human/blob/main/src/vk/vk_subscribers_research.ipynb)

In [1]:
import pandas as pd

csv_files = [
#    "https://raw.githubusercontent.com/componavt/sns4human/main/data/vk/posts/aparfenchikov_short.csv",
    "https://raw.githubusercontent.com/componavt/sns4human/main/data/vk/posts/aparfenchikov_category_short.csv",
#    "https://raw.githubusercontent.com/componavt/sns4human/main/data/vk/posts/aparfenchikov.csv",
]

df = pd.concat([pd.read_csv(url) for url in csv_files], ignore_index=True)
df = df.reset_index()  # make sure indexes pair with number of rows
df.head()

,index,id,category,text,date,likes,reposts,views
0,0,787797,образование,⚡Компания [https://vk.com/ksm.company?from=sea...,2024-11-01 09:30:00,118.0,10.0,5146.0
1,1,787790,культура,"👋Доброе утро, друзья! С удовольствием посмо...",2024-11-01 08:00:50,272.0,119.0,8301.0
2,2,787763,экономика,🧰 Бюджет Карелии в 2025 году останется ✔социал...,2024-10-31 20:30:17,84.0,22.0,7163.0
3,3,787753,война,🚛 Сегодня отправили в зону СВО подарки для наш...,2024-10-31 19:21:48,214.0,36.0,11431.0
4,4,787745,экономика,🖥 ✏Сегодня в режиме ВКС проведу заседание Прав...,2024-10-31 17:00:00,67.0,3.0,5220.0


In [2]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')


# import string
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
!pip install pymorphy3
import pymorphy3
morph = pymorphy3.MorphAnalyzer(lang='ru')

for index, row in df.iterrows():
    # print(index, row['text'])
    text = row['text'].lower()
    # print("lower text = ", text)
    tokens = nltk.word_tokenize(text, language='russian', preserve_line=True)

    normal_words=[]
    for word in tokens: # tokens in one line (one post)
      # no num and comma
      if (word.isalpha()) and (not word in stop_words):
        lemma = morph.parse(word)[0].normal_form
        # print("lemma = ", lemma)
        normal_words.append(lemma)
    n_title = ' '.join(normal_words)
    df.loc[index, 'normalized'] =n_title
    #print("normalized post = ", n_title)
print(df.loc[0, 'normalized'])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.4 MB/s eta 0:00:00
https ксм который построить школа суоярви получить разрешение ввод здание эксплуатация светлый класс просторный рекреация спортивный актовый зала лингафонный кабинет помещение продленка должный современный школа построить её средство нацпроект образование также школа петрозаводск деревянка кемь следующий важный этап планировать школа готовый начать учебный напроект образованиекарелия


In [4]:
# df["category"] = "" # let's empty column "category" to be filled
# df["category"] = ["политика", "медицина", "образование", "наука", "война", "спорт", "экономика", "история", "экология"]
# + Мир Общество События
print(df.columns)

# split the data into training and testing samples
from sklearn.model_selection import train_test_split
train_docs, test_docs = train_test_split(df, stratify=df.category, test_size=5, random_state=108, shuffle=True)

Index(['index', 'id', 'category', 'text', 'date', 'likes', 'reposts', 'views',
       'normalized'],
      dtype='object')


In [5]:
train_docs.shape, test_docs.shape

((6, 9), (5, 9))

In [6]:
pd.Series(test_docs.category).value_counts()

,count
category,
экономика,2
образование,1
война,1
культура,1


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=.2, min_df=3, max_features=2000)
doc_train_matrix = vectorizer.fit_transform(train_docs['normalized'])
words = vectorizer.get_feature_names()
doc_train_matrix

ValueError: max_df corresponds to < documents than min_df

In [ ]:
doc_test_matrix = vectorizer.transform(test_docs['normalized'])
doc_test_matrix

In [ ]:
topic_labels = ['Тема {}'.format(i) for i in range(1, 9)]


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=8, n_jobs=-1, max_iter=500,
                                learning_method='batch', evaluate_every=5,
                                verbose=1, random_state=42)
lda.fit(doc_train_matrix)

In [ ]:
from sklearn.externals import joblib
joblib.dump(lda, 'lda_500_iter.pkl')
lda = joblib.load('lda_500_iter.pkl')

In [ ]:
#Досліджуйте теми та розподіл слів
topics_count = lda.components_
print(topics_count.shape)
topics_count[:8]

In [ ]:
topics_prob = topics_count / topics_count.sum(axis=1).reshape(-1, 1)
topics = pd.DataFrame(topics_prob.T,
                      index=words,
                      columns=topic_labels)
topics.head()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
w = WordCloud(background_color='white',max_font_size = 50)
fig, axes = plt.subplots(ncols=2, nrows=4, figsize=(12, 12))
axes = axes.flatten()
for t, (topic, freq) in enumerate(topics.items()):
    w.generate_from_frequencies(freq.to_dict())
    axes[t].imshow(w, interpolation='bilinear')
    axes[t].set_title(topic, fontsize=12)
    axes[t].axis('off')
axes[5].set_visible(False)
plt.tight_layout()

In [ ]:
# всі слова мають позитивну ймовірність для всіх тем
topics[topics.gt(0).all(1)].shape[0] == topics.shape[0]

In [ ]:
n_words = 12
top_words = {}
for topic, words_ in topics.items():
    top_words[topic] = words_.nlargest(n_words).index.tolist()
pd.DataFrame(top_words)

In [ ]:
import seaborn as sns
sns.heatmap(topics, cmap='Blues')

In [ ]:
train_preds = lda.transform(doc_train_matrix)
train_preds.shape

In [ ]:
train_eval = pd.DataFrame(train_preds, columns=topic_labels, index=train_docs.category)
train_eval.head()

In [ ]:
train_eval.groupby(level='category').mean().plot.bar(title='Сер. Тема ймовірностей', figsize=(20,10));

In [ ]:
df = train_eval.groupby(level='category').idxmax(
    axis=1).reset_index(-1, drop=True)
plt.figure(figsize=(12, 12))
sns.heatmap(df.groupby(level='category').value_counts(normalize=True)
            .unstack(-1), annot=True, fmt='.1%', cmap='Blues', square=True)
plt.title('Дані навчання: Тематичні завдання')



In [ ]:
train_eval = pd.DataFrame(data=lda.transform(doc_train_matrix),
                          columns=topic_labels,
                          index=train_docs.category)
test_eval = pd.DataFrame(data=lda.transform(doc_test_matrix),
                         columns=topic_labels,
                         index=test_docs.category)

In [ ]:
import seaborn as sns

fig, axes = plt.subplots(ncols=2, figsize=(16,10))
source = ['Навчання', 'Тест']
for i, df in enumerate([train_eval, test_eval]):
    df = df.groupby(level='category').idxmax(
    axis=1).reset_index(-1, drop=True)
    sns.heatmap(df.groupby(level='category').value_counts(normalize=True)
            .unstack(-1), annot=True, fmt='.1%', cmap='Blues', square=True, ax=axes[i])
    axes[i].set_title('{} Дані: Тематичні завдання'.format(source[i]));

In [ ]:
#Перегляньте неправильно класифіковані статті
test_assignments = test_eval.groupby(level='category').idxmax(
    axis=1).reset_index(-1, drop=True).to_frame('predicted').reset_index()
test_assignments['title'] = test_docs.title.values
test_assignments.head()

In [ ]:
misclassified = test_assignments[(test_assignments.category == 'Політика') & (
    test_assignments.predicted == 'Тема 1')]
misclassified.title

In [ ]:
misclassified.title.tolist()

In [ ]:
!pip install pyLDAvis
import pyLDAvis.sklearn


In [ ]:
lda_viz = pyLDAvis.sklearn.prepare(lda, doc_train_matrix, vectorizer, mds='tsne')
pyLDAvis.display(lda_viz)